---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [3]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
def answer_one():
    return df['Gold'].idxmax()

answer_one()



'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [4]:
def answer_two():
    df_golds = df['Gold'].sub(df['Gold.1'], axis=0).idxmax()
    return df_golds

answer_two()


'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [5]:
def answer_three():
    df_gold = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]    
    df_diff = (df_gold['Gold'] - df_gold['Gold.1'])/df_gold['Gold.2']
    return df_diff.idxmax()

answer_three()



'Bulgaria'

### Question 4
Write a function to update the dataframe to include a new column called "Points" which is a weighted value where each gold medal counts for 3 points, silver medals for 2 points, and bronze mdeals for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [6]:
def answer_four():
    
    Points = (df['Gold'] * 3) + (df['Silver'] * 2) + (df['Bronze'] * 1)
    return Points
    
answer_four()


Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            898
Austria                              155
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              123
Belgium                              268
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             401
Burundi                                3
Cameroon                              12
Canada                               496
Chile                                 24
China                               1021
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [4]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [5]:
def answer_five():
    states_by_counties = census_df.groupby('STNAME')
    count_of_counties = states_by_counties['COUNTY'].count()
    return count_of_counties.idxmax()

answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)?

*This function should return a list of string values.*

In [15]:
def answer_six():
    newdf = census_df[census_df['SUMLEV']==50].copy()
    grouped = newdf.groupby('STNAME',as_index=False)
    ans = grouped['CENSUS2010POP'].agg(lambda x: sum(sorted(x,reverse= True)[0:3]))
    ans.sort_values(by='CENSUS2010POP',inplace=True,ascending = False)
    return list(ans['STNAME'][0:3])
answer_six()


states_by_counties = census_df[census_df['SUMLEV']== 50]
grouped_by_state = states_by_counties.groupby(['STNAME'], as_index=False)
summed = grouped_by_state.agg('sum')
#summed[:3]
#summed.reset_index(level=0, inplace=True)
summed.sort_values(by="CENSSUS2010POP"], inplace=True, ascending=False)





,STNAME,SUMLEV,REGION,DIVISION,STATE,COUNTY,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,Alabama,3350,201,402,67,4489,4779736,4780127,4785161,4801108,...,-227.514278,-203.888275,-146.497410,-114.965877,-131.203351,-186.330375,-156.743938,-100.911513,-63.278086,-77.431740
1,Alaska,1450,116,261,58,4340,710231,710249,714021,722720,...,-33.440736,-152.595613,-269.038219,-522.388655,-379.628897,63.103741,-47.900784,-168.074354,-411.285079,-264.650082
2,Arizona,750,60,120,60,208,6392017,6392307,6408208,6468732,...,6.340196,-28.516201,-36.708427,30.251698,-7.852474,25.060233,-7.708944,-15.666411,54.025924,16.927772
3,Arkansas,3750,225,525,375,5625,2915918,2915958,2922394,2938538,...,-187.219281,-370.382679,-322.504358,-415.673128,-329.528407,-155.311457,-338.123960,-287.209763,-374.162847,-286.827346
4,California,2900,232,522,348,3364,37253956,37254503,37334079,37700034,...,-246.189782,-222.243506,-200.860119,-117.589776,-43.237552,-126.118333,-103.733568,-74.557711,27.166256,104.229475


### Question 7
Which county has had the largest absolute change in population within the period 2010-2015 (hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns)?

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then it's largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [ ]:
def answer_seven():
    return "YOUR ANSWER HERE"

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [ ]:
def answer_eight():
    return "YOUR ANSWER HERE"